# Unrelated Script

In [91]:
import pandas as pd
import numpy as np
import spacy
sp_mod = spacy.load("en_core_web_sm")
import os, sys
import glob
from itertools import chain 

In [93]:
filenames = glob.glob('/Users/Mohit/Desktop/Work/Shreays Dataset/chuncker_raw_new_dataset/cnn/del/*.story', recursive = True)
# filenames = glob.glob('/Users/Mohit/Desktop/Work/Shreays Dataset/chuncker_raw_new_dataset/cnn/stories/*.story', recursive = True)
## filenames = filenames[0]
print(filenames)

['/Users/Mohit/Desktop/Work/Shreays Dataset/chuncker_raw_new_dataset/cnn/del/0a0a4c90d59df9e36ffec4ba306b4f20f3ba4acb.story']


In [94]:
files = {} 
allstory_list = []
for filename in filenames: 
    with open(filename, "r") as file: 
        if filename in files: 
            continue 
        files[filename] = file.read() 
for filename, text in files.items(): 
    allstory_list.append(text)

In [95]:
def convert_allstory_list(allstory_list):
    final_list = []
    for i in range(len(allstory_list)):
        res_list = allstory_list[0].splitlines()
        res = [ele for ele in res_list if ele != []]
        res = [x for x in res if x]
        # res = res[: len(res) - 8]
        final_list.append(res)
    return final_list
    # print(res)

In [96]:
final_list = convert_allstory_list(allstory_list)

In [97]:
# final_list = [['(CNN) -- Can a movie actually convince you to support torture? Can a movie really persuade you that "fracking" -- a process used to drill for natural gas -- is a danger to the environment? Can a movie truly cause you to view certain minority groups in a negative light?', 'How concerned is the gas industry?','So go ahead, enjoy these films and ones like them that are based on actual events or current hot issues. But while you are watching them, be aware you might be getting more than the price of ticket. You might also be getting a (mis)education.','In fact, there\'s no doubt that the studios behind these movies are overjoyed at the controversy that has erupted and the resulting free press. Indeed, the response of Sony Pictures to the uproar over "Zero Dark Thirty" tells you about what they really hope we will all do: "We encourage people to see the film before characterizing it."']]

In [98]:
# IMP
def convert_pairs(final_list):
    new_list = []
    for i in range(len(final_list)):
        for j in range(len(final_list[i])):
            new_sents = [] # for sentences with lenght greater than 5
            text = final_list[i][j]
            doc = sp_mod(text)
            # Tokenise the sentence
            sents = [elem for elem in doc.sents]

            if len(sents) > 1:

                # Idnetify the length of the sentence
                sents_len = [len(elem) for elem in doc.sents]
                
                # Ignore the tokenise setence with length less than 5 
                for k in range(len(sents_len)):
                    if sents_len[k] > 5:
                        new_sents.append(sents[k])
            else:
                new_sents = []
                
            # Assign to final list
            final_list[i][j] = new_sents

    final_list = list(chain.from_iterable(final_list))
    final_list = [x for x in final_list if x]
    new_list.append(final_list)

    return new_list

In [99]:
final_list_converted_pairs = convert_pairs(final_list)
# final_list_converted_pairs

In [100]:
sent_1_list = []
sent_2_list = []
for i in range(len(final_list_converted_pairs)):
    for j in range(len(final_list_converted_pairs[i])):
        first_val = final_list_converted_pairs[i][j][0]
        sent_1_list.append(first_val)
        
        last_val = final_list_converted_pairs[i][j][-1]
        sent_2_list.append(last_val)

# remove 1st and last element from sent_1_list and sent_2_list repsectively
sent_1_list = sent_1_list[1:] 
sent_2_list = sent_2_list[:-1]         
print(type(sent_1_list))

print("1st list: ", sent_1_list)
print("2nd list: ", sent_2_list)

<class 'list'>
1st list:  [Some scoff at the notion that movies do anything more than entertain., Anyone who doubts the potential influence that movies can have on public opinion need to look no further than two films that are causing an uproar even before they have opened nationwide., The first, "Zero Dark Thirty," is about the pursuit and killing of Osama bin Laden, which features scenes of torture., Critics of "Zero Dark Thirty" fear that audiences will accept as true the film's story line that torture was effective in eliciting information to locate bin Laden., Did torture really net bin Laden?, Opposition to the film escalated last week as three senior U.S. senators -- John McCain, Carl Levin and Dianne Feinstein -- sent a letter to the film's distributor, Sony Pictures, characterizing the film's use of torture as "grossly inaccurate and misleading., The hostility toward "Promised Land" shows us that it's not just politicians who complain about movie messages., It has set up a rap

In [101]:
columns = ["sent-1", "sent-2"]
final_df = pd.DataFrame(columns=columns)
final_df['sent-1'] = sent_1_list
final_df['sent-2'] = sent_2_list
final_df['sent-1'] = final_df['sent-1'].astype('str')
final_df['sent-2'] = final_df['sent-2'].astype('str')
# final_df

,sent-1,sent-2
0,Some scoff at the notion that movies do anythi...,Can a movie truly cause you to view certain mi...
1,Anyone who doubts the potential influence that...,And for those already agreeing with the film's...
2,"The first, ""Zero Dark Thirty,"" is about the pu...",They present hot button issues that manage to ...
3,"Critics of ""Zero Dark Thirty"" fear that audien...","The second, ""Promised Land,"" stars Matt Damon ..."
4,Did torture really net bin Laden?,They are rightfully concerned that the film wi...
5,Opposition to the film escalated last week as ...,Did torture really net bin Laden?
6,"The hostility toward ""Promised Land"" shows us ...",The senators bluntly informed Sony Pictures th...
7,It has set up a rapid response team to counter...,"Big business -- namely, the gas industry -- is..."
8,"Energy in Depth went as far as to ""fact check""...",These groups have scrutinized appearances by t...
9,This is the same reason that minority groups c...,"Within hours of Krasinski's appearance, Energy..."
